# Organize directories

The first step in training a ML model on sequences of images acquired from the ProtectCode app is to ensure the directory structure is set up for subsequent data loading. 

These commands assume the following: 

1) That the top directory for the training data and this notebook are both located in ~/src/YPB-AI/ on the EC2 instance. To get a zipped directory from S3 to that directory on your EC2 instance, you can run the following bash command: "aws s3 cp s3://your-bucket-name/yourZipFile.zip ~/src/YPB-AI/yourZipFile.zip". And to unzip it, you can run "unzip yourZipFile.zip"

2) That the jupyter notebook server was opened from the ~/src/ directory on the EC2 instance via the command "jupyter notebook --ip=0.0.0.0 --no-browser".

3) That underneath the top directory (which I usually call "Training" or something of the like), there are a series of subdirectories named and organized by classification label. For instance, each subdirectory that contains true negative image sequences, according to my implementation, contains the word "Negative". And subdirectories with true positives do not contain the word "Negative". 



In [19]:
import numpy as np
from glob import glob
import os, sys
import re

Set the name of your top data directory below. 

In [13]:
# topDirName = 'Training'
topDirName = '/home/ubuntu/src/YPB-AI/23_11_19_PB'

Lump together all subdirectories with the same classification label. 

In [23]:
#Assumes current working directory is ~/src/YPB-AI
allSamples = np.array(glob(topDirName + '/*'))
os.system('mkdir ' + topDirName + '/Absent')
os.system('mkdir ' + topDirName + '/Present')

for sample in allSamples: 
    if 'Negative' in sample or 'Sat' in sample:
        cmdString = "mv " + sample + " " + topDirName + "/Absent"
    else:
        cmdString = "mv " + sample + " " + topDirName + "/Present"
    os.system(cmdString)

Each directory within the classification subdirectory contains a series of folders that have the following contents:

1) A .txt file containing some timing and other metadata about the app run.

2) A folder called 'Photos' which contains the sequence of images acquired.

The next cell will obliterate the middle directory (the directory just below the classification directory) and repackage each run's photos into a uniquely identifiable folder of its own (lacking the text file) at the level where the middle directories once sat.



In [25]:
# Find 'vestigial' directories to be deleted after unpacking.
presentVestDirs = np.array(glob(topDirName + '/Present/*'))
absentVestDirs = np.array(glob(topDirName + '/Absent/*'))

# Unpack the JPG photo files into identifiable directories.
allPresentSamples = np.array(glob(topDirName + '/Present/*/*'))
for sample in allPresentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    # digestedTimestamp = re.split('_',digestedSample[3])
    digestedTimestamp = re.split('_',digestedSample[8])
    # newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    newDir = digestedSample[7] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    
allAbsentSamples = np.array(glob(topDirName + '/Absent/*/*'))
for sample in allAbsentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    # digestedTimestamp = re.split('_',digestedSample[3])
    digestedTimestamp = re.split('_',digestedSample[8])
    # newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    newDir = digestedSample[7] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    
    
# Delete vestigial directories.
for direc in presentVestDirs:
    os.system('rm -rf ' + direc)
    
for direc in absentVestDirs:
    os.system('rm -rf ' + direc)

Look through all directories and find image sequences containing the file "IMG_9999.JPG". If found, sort and rename to an appropriate image sequence. This is necessary because when iOS reaches "IMG_9999.JPG" it wraps back around to "IMG_0000.JPG" for the next in the sequence. If this is left as is, the sorting operation that follows will sort the images into the incorrect timing order. 

In [21]:
print(digestedSample)
print(digestedTimestamp)
print(allAbsentSamples)

['', 'home', 'ubuntu', 'src', 'YPB-AI', '23_11_19_PB', 'Absent', '7_filledTube_Negative', 'ProtectCodeScannerResults_05-Nov-2019_15-51-14']
['src']
['/home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_filledTube_Negative/ProtectCodeScannerResults_05-Nov-2019_15-51-14'
 '/home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_filledTube_Negative/ProtectCodeScannerResults_05-Nov-2019_15-21-54'
 '/home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_filledTube_Negative/ProtectCodeScannerResults_05-Nov-2019_15-43-59'
 '/home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_filledTube_Negative/ProtectCodeScannerResults_05-Nov-2019_15-30-00'
 '/home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_filledTube_Negative/ProtectCodeScannerResults_05-Nov-2019_15-26-59'
 '/home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_filledTube_Negative/ProtectCodeScannerResults_05-Nov-2019_15-44-30'
 '/home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_filledTube_Negative/ProtectCodeScannerResults_05-Nov-2019_16-14-24'
 '/home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_

In [30]:
print(digestedSample)

['.', 'Absent', 'XS_emptyTube_Negative_05-Nov-2019_17-22-59', 'IMG_9999.JPG']


In [38]:
digestedSample

['.',
 'Absent',
 '7_greenLabel_Spaced_Negative_06-Nov-2019_15-16-33',
 'IMG_9999.JPG']

In [39]:
# Get the filenames.
# imgSeqsToCorrect = os.popen('cd ~/src/YPB-AI/; find -name "*IMG_9999.JPG"').read()
imgSeqsToCorrect = os.popen('cd ~/src/YPB-AI/23_11_19_PB/; find -name "*IMG_9999.JPG"').read()
# Split into unique entities.
imgSeqsToCorrect = re.split('\n',imgSeqsToCorrect)
# Drop the last entry, it's empty.
imgSeqsToCorrect.pop();

# Change any files that don't start with 'IMG_9' (i.e., any files that come after the wraparound) to start with 'J'.
# This will mean that when they are sorted, they will come correctly after the wraparound.
for sample in imgSeqsToCorrect:
    digestedSample = re.split('/',sample)
    # pathToDir = os.path.join(digestedSample[0],digestedSample[1],digestedSample[2],digestedSample[3])
    pathToDir = os.path.join(topDirName,digestedSample[1],digestedSample[2])
    # pathToDir = topDirName + '/' + digestedSample[0] + '/' + digestedSample[1] + '/' + digestedSample[2] + '/' + digestedSample[3]
    try:
        filesInDir = os.listdir(pathToDir)
        for i in range(len(filesInDir)):
            if 'IMG_9' not in filesInDir[i]:
                newFile = filesInDir[i].replace('I','J',1)
                # cmdString = 'mv ' + os.path.join(pathToDir,filesInDir[i]) + ' ' + os.path.join(pathToDir,newFile)
                cmdString = 'mv ' + os.path.join(pathToDir,filesInDir[i]) + ' ' + os.path.join(pathToDir,newFile)
                # cmdString = 'mv ' + '/' + filesInDir[i] + '/' + ' ' + pathToDir + '/' + newFile
                os.system(cmdString)
    except Exception as e:
        print(e)

# Use a CNN to transform image sequences to a pickle file and organize the results.

In [40]:
import torch
import torchvision.models as models
from PIL import Image
import pickle

Use a pretrained CNN model (here using DenseNet121 because it has worked well in the past) with the final classifier layer chopped off.

In [41]:
# Import the CNN model and move it to CUDA if available.
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
del model.classifier

if torch.cuda.is_available():
    model.to('cuda');

Define the function that will transform model output into a pickle file.

In [42]:
def transform_to_pickle(model,topdir):
    model.eval()
    ndx = 0
    topdirs = [direc for direc in os.listdir(topdir) if direc[0] is not '.']
    
    # Calculate the total number of stacks to transform.
    numStacks = 0
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        numStacks += len(directories)
    
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        
        
        # Do the transformation.
        for photoDir in directories:
            
            thisPhotoDir = os.path.join(thisClassDir, photoDir)
            ##pin pin fix##
            
#             print(thisPhotoDir)
#             print(os.path.isfile(photoDir))
#             continue
            
            if not os.path.isdir(thisPhotoDir):
                continue

            ## end ##
            files = [photofile for photofile in os.listdir(thisPhotoDir) if photofile.lower().endswith('jpg')]
            print("Transforming image stack {}".format(thisPhotoDir))
            files.sort()
            output = []
            # Make sure to appropriately transform the image size and normalize the RGB values, as the model expects. 
            for file in files:
                thisFile = os.path.join(thisPhotoDir,file)
                imgArr = np.float32(np.array(Image.open(thisFile).resize((224,224))).reshape([1,3,224,224]))
                imgArr[0][0] = ((imgArr[0][0] - 123.675)/58.395)
                imgArr[0][1] = ((imgArr[0][1] - 116.28)/57.12)
                imgArr[0][2] = ((imgArr[0][2] - 103.53)/57.375)
                if torch.cuda.is_available():
                    output.append(model(torch.cuda.FloatTensor(imgArr)))
                else:
                    output.append(model(torch.FloatTensor(imgArr)))

            FinalOutput = torch.FloatTensor(len(files),1,1024)
            for i in range(len(files)):
                FinalOutput[i] = output[i]
            pathStr = thisPhotoDir + '.pickle'
            with open(pathStr,'wb') as f:
                pickle.dump(FinalOutput,f)
                ndx += 1
                print("{}% complete.".format(ndx/numStacks*100))
    

Do the transformation

In [43]:
transform_to_pickle(model,topDirName)

Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_redBox_Spaced_Negative_06-Nov-2019_12-01-55
0.07283321194464676% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/6S_filledTube_Negative_05-Nov-2019_14-20-48
0.14566642388929352% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/XS_filledTube_Negative_05-Nov-2019_16-38-10
0.21849963583394028% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/XS_filledTube_Negative_05-Nov-2019_16-47-54
0.29133284777858703% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_filledTube_Negative_05-Nov-2019_15-36-36
0.3641660597232338% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/XS_emptyTube_Negative_05-Nov-2019_17-19-28
0.43699927166788055% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_greenLabel_Spaced_Negative_06-Nov-2019_16-12-34
0.5098324836125273% compl

4.224326292789512% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_redBox_Tight_Negative_06-Nov-2019_10-28-57
4.297159504734159% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_greenLabel_Spaced_Negative_06-Nov-2019_15-53-34
4.369992716678805% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_redBox_Spaced_Negative_06-Nov-2019_11-16-53
4.442825928623452% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_greenLabel_Spaced_Negative_06-Nov-2019_14-39-51
4.515659140568099% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/6S_filledTube_Negative_05-Nov-2019_14-09-48
4.588492352512746% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/6S_filledTube_Negative_05-Nov-2019_14-47-15
4.6613255644573925% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_redBox_Angle_Negative_06-Nov-2019_13-43

8.448652585579024% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/6S_filledTube_Negative_05-Nov-2019_14-33-01
8.52148579752367% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/XS_filledTube_Negative_05-Nov-2019_16-59-10
8.594319009468318% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_redBox_Angle_Negative_06-Nov-2019_14-12-38
8.667152221412964% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/6S_filledTube_Negative_05-Nov-2019_14-37-09
8.73998543335761% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_greenLabel_Spaced_Negative_06-Nov-2019_16-23-50
8.812818645302258% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_filledTube_Negative_05-Nov-2019_15-38-45
8.885651857246904% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_redBox_Spaced_Negative_06-Nov-2019_12-03-26
8.95848

12.672978878368536% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/6S_filledTube_Negative_05-Nov-2019_14-36-49
12.745812090313184% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_filledTube_Negative_05-Nov-2019_16-11-33
12.818645302257831% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/XS_filledTube_Negative_05-Nov-2019_16-37-58
12.891478514202475% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_redBox_Spaced_Negative_06-Nov-2019_11-13-26
12.964311726147123% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_redBox_Angle_Negative_06-Nov-2019_14-06-32
13.03714493809177% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_greenLabel_Spaced_Negative_06-Nov-2019_15-06-56
13.109978150036417% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/XS_emptyTube_Negative_05-Nov-2019_17-26-37


16.89730517115805% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_redBox_Angle_Negative_06-Nov-2019_12-26-53
16.970138383102697% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_redBox_Tight_Negative_06-Nov-2019_10-16-14
17.04297159504734% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/XS_filledTube_Negative_05-Nov-2019_16-43-31
17.11580480699199% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_redBox_Tight_Negative_06-Nov-2019_10-27-12
17.188638018936636% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_greenLabel_Spaced_Negative_06-Nov-2019_16-25-28
17.26147123088128% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_filledTube_Negative_05-Nov-2019_15-21-40
17.33430444282593% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Absent/7_redBox_Tight_Negative_06-Nov-2019_10-10-47
17

...

81.35469774217043% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/XS_filledTube_05-Nov-2019_09-43-15
81.42753095411508% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/6S_filledTube_05-Nov-2019_11-05-13
81.50036416605973% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/XS_filledTube_05-Nov-2019_09-53-29
81.57319737800437% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/7_fiiledTube_05-Nov-2019_13-21-58
81.64603058994902% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/6S_filledTube_05-Nov-2019_11-08-46
81.71886380189366% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/XS_emptyTube_05-Nov-2019_17-10-29
81.7916970138383% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/6S_filledTube_05-Nov-2019_11-01-08
81.86453022578296% complete.
Transforming image stack /home/ubuntu/src/Y

85.87035688273853% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/6S_filledTube_05-Nov-2019_11-13-56
85.94319009468317% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/XS_filledTube_05-Nov-2019_10-12-13
86.01602330662782% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/XS_filledTube_05-Nov-2019_10-15-22
86.08885651857247% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/7_fiiledTube_05-Nov-2019_12-33-32
86.16168973051711% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/6S_filledTube_05-Nov-2019_10-46-37
86.23452294246177% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/XS_filledTube_05-Nov-2019_10-06-36
86.3073561544064% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/7_fiiledTube_05-Nov-2019_13-20-02
86.38018936635106% complete.
Transforming image stack /home/ubuntu/src/Y

90.38601602330662% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/7_fiiledTube_05-Nov-2019_12-47-24
90.45884923525126% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/6S_filledTube_05-Nov-2019_10-39-39
90.53168244719592% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/6S_filledTube_05-Nov-2019_11-19-46
90.60451565914057% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/XS_filledTube_05-Nov-2019_09-43-38
90.67734887108521% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/XS_filledTube_05-Nov-2019_10-07-02
90.75018208302986% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/6S_filledTube_05-Nov-2019_11-15-34
90.82301529497451% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/7_fiiledTube_05-Nov-2019_12-32-09
90.89584850691915% complete.
Transforming image stack /home/ubuntu/src/

94.90167516387473% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/6S_filledTube_05-Nov-2019_11-18-02
94.97450837581938% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/XS_filledTube_05-Nov-2019_10-04-35
95.04734158776402% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/7_fiiledTube_05-Nov-2019_13-29-04
95.12017479970866% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/7_fiiledTube_05-Nov-2019_12-46-16
95.19300801165332% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/7_fiiledTube_05-Nov-2019_13-11-18
95.26584122359796% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/6S_filledTube_05-Nov-2019_10-59-38
95.3386744355426% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/XS_filledTube_05-Nov-2019_09-43-50
95.41150764748726% complete.
Transforming image stack /home/ubuntu/src/YP

99.41733430444283% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/7_fiiledTube_05-Nov-2019_12-35-24
99.49016751638747% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/7_fiiledTube_05-Nov-2019_12-54-14
99.56300072833211% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/7_fiiledTube_05-Nov-2019_12-33-16
99.63583394027677% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/XS_emptyTube_05-Nov-2019_17-11-45
99.70866715222141% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/7_fiiledTube_05-Nov-2019_13-24-33
99.78150036416606% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/6S_filledTube_05-Nov-2019_11-12-23
99.8543335761107% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/23_11_19_PB/Present/XS_filledTube_05-Nov-2019_09-44-42
99.92716678805536% complete.
Transforming image stack /home/ubuntu/src/YPB-

Clean up the directory, removing everything but the pickle files.

In [44]:
# Assemble all the pickle files and remove the rest.
classDirs = [direc for direc in os.listdir(topDirName) if direc[0] is not '.']
for classDir in classDirs:
    newDir = os.path.join(topDirName,classDir)
    fileDirs = [direc for direc in os.listdir(newDir) if direc[0] is not '.' and not direc.lower().endswith('.pickle')]
    for fileDir in fileDirs:
        pickleFileDir = os.path.join(newDir,fileDir)
        os.system('rm -rf ' + pickleFileDir)

# Split the directory into train/test/validation sets

In [45]:
import random

In [47]:
# finalDirName = 'Split_Data'
finalDirName = 'Split_Data_23_11_19'

In [48]:
def split_folders(inputDir, outputDir, trainValTestRatio, seed):
    # Assume inputDir is organized as 'inputDir/class/topFolder/photos'
    classes = os.listdir(inputDir)
    # Exclude folders where '.' is the first character.
    classes = [directory for directory in classes if directory[0] != '.']
    numClasses = len(classes)
    # Create output directories
    os.system('mkdir ' + outputDir)
    os.system('mkdir ' + outputDir + '/train')
    os.system('mkdir ' + outputDir + '/valid')
    os.system('mkdir ' + outputDir + '/test')
    # Pull files and put into the appropriate folders.
    for directory in classes:
        files = os.listdir(os.path.join(inputDir, directory))
        files.sort()
        random.seed(seed)
        random.shuffle(files)
        numFiles = len(files)
        # Get the indices.
        ndxAmts = np.intc((np.round(trainValTestRatio / np.sum(trainValTestRatio) * numFiles)))
        numNdxes = np.sum(ndxAmts)
        if numNdxes != numFiles:
            ndxAmts[0] += numFiles-numNdxes
        trainFiles = files[0:ndxAmts[0]]
        valFiles = files[ndxAmts[0]:ndxAmts[0]+ndxAmts[1]]
        testFiles = files[ndxAmts[0]+ndxAmts[1]:]
        #Put the training directories in the appropriate folder
        # First make the class directories.
        os.system('mkdir ' + outputDir + '/train/' + directory)
        os.system('mkdir ' + outputDir + '/valid/' + directory)
        os.system('mkdir ' + outputDir + '/test/' + directory)
        for file in trainFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/train/' + directory + '/' + file)
        for file in valFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/valid/' + directory + '/' + file)
        for file in testFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/test/' + directory + '/' + file)

In [49]:
split_folders(topDirName, finalDirName,[0.8,0.1,0.1],7778)